<h1> Term Project </h1>

<h2> 주제 : 서울 근방 지역 데이트 코스 추천 알고리즘 </h2>

## (1) 주제 선정 이유<br>
자신의 거주지 이외의 지역에 여행 갔을 때, 누구나 먹거리, 놀거리 등을 따로 따로 검색한다. 실제로 본인은 대학교 입학 전까지 서울에 가 본 적이 없었는데, 갈 때마다 노는 경로를 정하는 것에 어려움을 겪었다. 이번 프로젝트의 목적은 이와 같은 수고를 덜어주기 위한 것이다. 서울 지역 내에서 어떤 순서로 놀 지를 정하면 저장된 위치 정보를 기반으로 최적 경로 알고리즘을 이용해서 경로를 추천하여, 수고를 덜겠다.



## (2) 가설 정의 : 
<br>
- 먹거리 지점(위도, 경도를 통해 표현)<br>
- 놀거리 지점(위도, 경도를 통해 표현)<br>
- 카페 지점(위도, 경도를 통해 표현)<br>
<br>

위와 같은 데이터 정보를 가상으로 만들고, 최적의 이동경로는 어떤 것인지 찾아보고 싶다.

## (3) 가상의 위치를 설정
<br>
가상의 위도와 경도를 기준으로 위치를 설정하여, csv파일에 저장해두겠다.<br>
다음 코드를 돌며, 서울 위도, 경도 범위 내에서 random한 가상의 값들을 만들어낼 것이다.<br>
이를 Place_food.csv, Place_cafe.csv, Place_doing_fun.csv 파일에 저장해두었다.

In [ ]:
import random

# 랜덤한 위도 및 경도 값 생성 (예시로 30개의 위치 정보 생성)
locations = []
for _ in range(30):
    latitude = round(random.uniform(37.5, 38.0), 6)  # 서울 위도 범위
    longitude = round(random.uniform(126.8, 127.2), 6)  # 서울 경도 범위
    locations.append((latitude, longitude))

# 생성된 위치 정보 출력
print(locations)

## (4) 구현을 위한 데이터 설정
<br>
Base class인 Place와 이것의 Derived class인 Restaurant, Cafe, Entertainments이라는 클래스를 만들어보겠다.

In [ ]:
class Place:

    def __init__(self, name, location, type, rating, operating_time):
        self.name = name
        self.location = location
        self.type = type
        self.rating = rating
        self.operating_tiem = operating_time

    def getName(self):
        return self.name()
    def getLocation(self):
        return self.location()
    def getType(self):
        return self.type
    def getRating(self):
        return self.rating
    def getOperatingTime(self):
        return self.operating_time


class Restaurant(Place):

    def __init__(self, name, location, type, rating, operating_time, type_of_foods, rep_menu, avg_price):
        super().__init__(name, location, type, rating, operating_time)
        self.types_of_foods = type_of_foods
        self.rep_menu = rep_menu
        self.avg_price = avg_price

    def getTypeOfFood(self):
        return self.types_of_foods
    def getRepMenu(self):
        return self.rep_menu
    def getAvgPrice(self):
        return self.avg_price
    

class Cafe(Place):

    def __init__(self, name, location, type, rating, operating_time,no_coffee_possible,rep_menu):
        super().__init__(name, location, type, rating, operating_time)
        self.no_coffee_possible = no_coffee_possible
        self.rep_menu = rep_menu

    def getNoCoffeePossible(self):
        return self.no_coffee_possible
    def getRepMenu(self):
        return self.rep_menu


class Entertainments(Place):

    def __init__(self, name, location, type, rating, operating_time, performace_available, entertainments_info):
        super().__init__(name, location, type, rating, operating_time)
        self.perforamance_available = performace_available
        self.entertainments_info = entertainments_info
        
    def getPerformanceAvailable(self):
        return self.perforamance_available
    def getEntertainmentsInfo(self):
        return self.entertainments_info

## (5) 필요한 함수 구현 그리고 필요한 library
**최근접 이웃 알고리즘**을 사용할 것이다. 특정 위치에서 다음에 갈 수 있는 위치 중 제일 가까운 곳으로 가게 되고, 그 다음 위치에서 다시 같은 방식으로 다음 위치를 정한다.<br><br>
1. distance_bet_places(#present_places, #next_places) : #present_place, #next_place는 각각 위치의 이름과 해당 위치의 위도 경도(십진수) 값들이다. 현재 있을 장소들 각각에서 다음 장소가 될 만한 곳까지의 거리를 모두 구하고, 이들을 dict에 넣어서 return한다.<br>
2. make_Full_course(#i,#distance_bet_places,#Full_course_candidates)
* #i : 이전에 얼마만큼의 수행이 있었는 지를 알려주는 변수이다. 
* #distance_bet_places :  1번 함수의 수행 결과를 의미한다.
* #Full_course_candidates : 최종적인 후보군들을 저장하는 변수이다. list형식으로 되어있다.
* 서로 다른 위치들을 포함해야하니, 같은 장소가 중복되지 않게 설정한다.
<br>

**-3rd_party library List**

1. **geopy** => pip install geopy
2. **folium** => pip install folium (찍힌 위치를 지도 위에 시각화 해서 보기 위해 쓰인다.)
3. **pandas** => pip install pandas

In [ ]:
def distance_bet_places(Collection_place_now,Collection_place_next):
    distance_bet_places = dict()
    for place_now in Collection_place_now:
        dist_candidate = dict()
        for place_next in Collection_place_next:
            dist_candidate[place_next] = geodesic(Collection_place_now[place_now],Collection_place_next[place_next]).meters
        distance_bet_places[place_now] = dist_candidate

    return distance_bet_places

def make_Full_course(i,distance_bet_places,Full_course_candidates):
    l = list()
    if i ==0:
        for place in distance_bet_places:
            present_place, next_places = (place, distance_bet_places[place])
            for candidate in next_places:         
                for candidate in next_places:
                    if next_places[candidate] != 0:
                        l.append([[present_place,candidate],next_places[candidate]])

        l.sort(key=lambda x: x[1])
        idx = 1
        var = l[0][0][0]
        while len(Full_course_candidates)<5:
            if l[idx][0][0] != var:
                Full_course_candidates.append(l[idx])
                var = l[idx][0][0]
            idx +=1


    else:
        for place in distance_bet_places:
            present_place, next_places = (place, distance_bet_places[place])
            for path in Full_course_candidates:
                for candidate in next_places:
                    if path[0][-1] == present_place:
                        l.append([[present_place,candidate],next_places[candidate]])
        l.sort(key=lambda x: x[1])

        for path in Full_course_candidates:
            for sub_path in l:
                if sub_path[0][0]==path[0][-1] and sub_path[0][1] not in path[0]:
                    path[0].append(sub_path[0][1])
                    path[1] += sub_path[1]
                    break
            
    return Full_course_candidates

## (6) Outline_Of_Process
1. <br> while문을 돌며, 시작 입력이 입력이 제대로 주어지면 반복문을 돌며 계속 프로그램을 실행하도록 한다.<br>

2. <br>main 코드에서 str 형식의 코스를 입력을 받을 것이다. (ex "식사->카페->놀거리->놀거리->식사" ) <br>

3. <br>입력받은 양식에 맞추어, csv파일로부터 정보를 읽어온다.<br>

4. <br>구현한 함수를 돌며, 최근접 이웃 알고리즘을 수행하며, 5가지 추천 코스를 출력한다.<br>

5. <br>결정된 코스 5개를 folium module을 이용해, 지도 위에 시각화 한다. 각각의 위치에 커서를 올리면, 이곳이 어느 코스의 무슨 위치인지 보여준다.<br>

6. <br>관심이 가는 코스에 대한 추가적인 정보를 요청하면, 사용자가 '종료'를 입력할 때까지 계속해서 수행한다.<br>

7. <br>사용자가 종료를 원한다면 '종료'를 입력받고, 프로그램을 종료한다.<br>

## (7) 전체 프로그램 코드

In [4]:
import pandas as pd
import csv
from geopy.distance import geodesic
import folium
import webbrowser

class Place:

    def __init__(self, name, location, type, rating, operating_time):
        self.name = name
        self.location = location
        self.type = type
        self.rating = rating
        self.operating_tiem = operating_time

    def getName(self):
        return self.name
    def getLocation(self):
        return self.location
    def getType(self):
        return self.type
    def getRating(self):
        return self.rating
    def getOperatingTime(self):
        return self.operating_time


class Restaurant(Place):

    def __init__(self, name, location, type, rating, operating_time, type_of_foods, rep_menu, avg_price):
        super().__init__(name, location, type, rating, operating_time)
        self.types_of_foods = type_of_foods
        self.rep_menu = rep_menu
        self.avg_price = avg_price

    def getTypeOfFood(self):
        return self.types_of_foods
    def getRepMenu(self):
        return self.rep_menu
    def getAvgPrice(self):
        return self.avg_price
    

class Cafe(Place):

    def __init__(self, name, location, type, rating, operating_time,no_coffee_possible,rep_menu):
        super().__init__(name, location, type, rating, operating_time)
        self.no_coffee_possible = no_coffee_possible
        self.rep_menu = rep_menu

    def getNoCoffeePossible(self):
        return self.no_coffee_possible
    def getRepMenu(self):
        return self.rep_menu


class Entertainments(Place):

    def __init__(self, name, location, type, rating, operating_time, performance_available, entertainments_info):
        super().__init__(name, location, type, rating, operating_time)
        self.performance_available = performance_available
        self.entertainments_info = entertainments_info
        
    def getPerformanceAvailable(self):
        return self.performance_available
    def getEntertainmentsInfo(self):
        return self.entertainments_info

def make_Full_course(i,distance_bet_places,Full_course_candidates):
    l = list()
    if i ==0:
        for place in distance_bet_places:
            present_place, next_places = (place, distance_bet_places[place])
            for candidate in next_places:         
                for candidate in next_places:
                    if next_places[candidate] != 0:
                        l.append([[present_place,candidate],next_places[candidate]])

        l.sort(key=lambda x: x[1])
        idx = 1
        var = l[0][0][0]
        while len(Full_course_candidates)<5:
            if l[idx][0][0] != var:
                Full_course_candidates.append(l[idx])
                var = l[idx][0][0]
            idx +=1


    else:
        for place in distance_bet_places:
            present_place, next_places = (place, distance_bet_places[place])
            for path in Full_course_candidates:
                for candidate in next_places:
                    if path[0][-1] == present_place:
                        l.append([[present_place,candidate],next_places[candidate]])
        l.sort(key=lambda x: x[1])

        for path in Full_course_candidates:
            for sub_path in l:
                if sub_path[0][0]==path[0][-1] and sub_path[0][1] not in path[0]:
                    path[0].append(sub_path[0][1])
                    path[1] += sub_path[1]
                    break
            
    return Full_course_candidates

def distance_bet_places(Collection_place_now,Collection_place_next):
    distance_bet_places = dict()
    for place_now in Collection_place_now:
        dist_candidate = dict()
        for place_next in Collection_place_next:
            dist_candidate[place_next] = geodesic(Collection_place_now[place_now],Collection_place_next[place_next]).kilometers
        distance_bet_places[place_now] = dist_candidate

    return distance_bet_places


if __name__ == "__main__":
    command = input("안녕하세요! 시작을 원하시면 '시작'을, 종료하시려면 '종료'를 입력해주세요! \n(⋇ space bar가 포함되면 안 됩니다.) \n")
    while(command!="종료"):
        date = input("원하는 데이트 코스를 입력해주세요.\n(⋇ space bar가 포함되면 안 됩니다.)\n(선택지 : 식사, 카페, 놀거리)\n ex)식사->카페->놀거리->카페")
        separted_date = date.split('->')

        flag = False
        if len(separted_date)<=1:
            print("코스는 2개 이상으로 구성되어야 합니다! 주의 사항에 맞춰 다시 입력해주세요!")
            print('--------------------------------------------------------------------------------')
            flag = True
        else:
            for item in separted_date:
                if " " in item:
                    print("space bar가 포함됐습니다! 주의 사항에 맞춰 다시 입력해주세요!")
                    print('--------------------------------------------------------------------------------')
                    flag = True
                elif item != "식사" and item != "놀거리" and item != "카페":
                    print("선택지에 없는 코스 항목이 입력되었습니다! 주의 사항에 맞춰 다시 입력해주세요!")
                    print('--------------------------------------------------------------------------------')
                    flag = True

        if flag:
            continue
        
        file_path_food = 'Place_food.csv'
        file_path_cafe = 'Place_cafe.csv'
        file_path_doing_fun = 'Place_doing_fun.csv'
      
        Restaurants = list()

        with open(file_path_food, newline='') as csvfile:
            Collection_restaurant = csv.reader(csvfile)
            Collection_restaurant.__next__()
            for info in Collection_restaurant: 
                info[1] = info[1].split(',')
                info[1] = (float(info[1][0]), float(info[1][1]))
                Restaurants.append(Restaurant(info[0],info[1],info[2],info[3],info[4],info[5],info[6],info[7]))
                    
        
        Doing_funs = list()

        with open(file_path_doing_fun, newline='') as csvfile:
            Collection_doing_fun = csv.reader(csvfile)
            Collection_doing_fun.__next__()
            for info in Collection_doing_fun:
                info[1] = info[1].split(',')
                info[1] = (float(info[1][0]), float(info[1][1]))
                Doing_funs.append(Entertainments(info[0],info[1],info[2],info[3],info[4],info[5],info[6]))


        p = input("커피 못 드시나요?(space bar가 포함되면 안 됩니다.)\n(선택지 : yes or no)\n ex)yes")
        if p == "yes":
            coffee = False
        else:
            coffee = True
        
        Cafes = list()

        with open(file_path_cafe, newline='') as csvfile:
            Collection_cafe = csv.reader(csvfile)
            Collection_cafe.__next__()
            for info in Collection_cafe:
                info[1] = info[1].split(',')
                info[1] = (float(info[1][0]), float(info[1][1]))
                Cafes.append(Cafe(info[0],info[1],info[2],info[3],info[4],info[5],info[6]))

        Full_course_candidates =list()
        num = len(separted_date)
        

        dist_from_food_to_cafe = dict()
        dist_from_food_to_doing_fun = dict()
        dist_from_food_to_food = dict()

        dist_from_cafe_to_food = dict()
        dist_from_cafe_to_doing_fun = dict()
        dist_fromm_cafe_to_cafe = dict()

        dist_from_doing_fun_to_food = dict()
        dist_from_doing_fun_to_cafe = dict()
        dist_from_doing_fun_to_doing_fun = dict()


        for i in [Restaurants, Cafes, Doing_funs]:
            for j in [Restaurants, Cafes, Doing_funs]:
                Collection_place_now = dict()
                Collection_place_next = dict()

                for places_now in i:
                    Collection_place_now[places_now.getName()] = places_now.getLocation()
                for places_next in j:
                    Collection_place_next[places_next.getName()] = places_next.getLocation()

                if i == Restaurants:
                    if j == Restaurants:
                        dist_from_food_to_food = distance_bet_places(Collection_place_now,Collection_place_next)
                    if j == Cafes:
                        dist_from_food_to_cafe = distance_bet_places(Collection_place_now,Collection_place_next)
                    if j == Doing_funs:
                        dist_from_food_to_doing_fun = distance_bet_places(Collection_place_now,Collection_place_next)

                if i == Cafes:
                    if j == Restaurants:
                        dist_from_cafe_to_food = distance_bet_places(Collection_place_now,Collection_place_next)
                    if j == Cafes:
                        dist_from_cafe_to_cafe = distance_bet_places(Collection_place_now,Collection_place_next)
                    if j == Doing_funs:
                        dist_from_cafe_to_doing_fun = distance_bet_places(Collection_place_now,Collection_place_next)
                if i == Doing_funs:
                    if j == Restaurants:
                        dist_from_doing_fun_to_food = distance_bet_places(Collection_place_now,Collection_place_next)
                    if j == Cafes:
                        dist_from_doing_fun_to_cafe = distance_bet_places(Collection_place_now,Collection_place_next)
                    if j == Doing_funs:
                        dist_from_doing_fun_to_doing_fun = distance_bet_places(Collection_place_now,Collection_place_next)


        for i in range(len(separted_date)-1):
            Collection_place_now = dict()
            Collection_place_next = dict()

            match separted_date[i]:
                case "식사":
                    match separted_date[i+1]:
                        case "식사":
                            Full_course_candidates = make_Full_course(i,dist_from_food_to_food,Full_course_candidates)
                        case "카페":
                            Full_course_candidates = make_Full_course(i,dist_from_food_to_cafe,Full_course_candidates)
                        case "놀거리":
                            Full_course_candidates = make_Full_course(i,dist_from_food_to_doing_fun,Full_course_candidates)

                case "카페":
                    match separted_date[i+1]:
                        case "식사":
                            Full_course_candidates = make_Full_course(i,dist_from_cafe_to_food,Full_course_candidates)
                        case "카페":
                            Full_course_candidates = make_Full_course(i,dist_from_cafe_to_cafe,Full_course_candidates)
                        case "놀거리":
                            Full_course_candidates = make_Full_course(i,dist_from_cafe_to_doing_fun,Full_course_candidates)

                case "놀거리":
                    match separted_date[i+1]:
                        case "식사":
                            Full_course_candidates = make_Full_course(i,dist_from_doing_fun_to_food,Full_course_candidates)
                        case "카페":
                            Full_course_candidates = make_Full_course(i,dist_from_doing_fun_to_cafe,Full_course_candidates)
                        case "놀거리":
                            Full_course_candidates = make_Full_course(i,dist_from_doing_fun_to_doing_fun,Full_course_candidates)



        df = pd.DataFrame(Full_course_candidates, columns=['데이트 코스','예상 도보 시간(h)(3km/h기준)'], index=['1번째 추천 : ', '2번째 추천 : ','3번째 추천 : ','4번째 추천 : ','5번째 추천 : '])
        max_len = df['데이트 코스'].str.len().max()
        df["데이트 코스"] = df["데이트 코스"].apply(' -> '.join)
        df['데이트 코스'] = df["데이트 코스"].apply(lambda x: x.ljust(max_len))

        print(df)

        for Course in Full_course_candidates:
            Course[1] = Course[1]/3  #(거리)/3km/h
        routes = dict()
        n = 1
        for Course in Full_course_candidates:
            line = list()
            for idx in range(len(Course[0])):
                match separted_date[idx]:
                    case "식사":
                        for restaurant in Restaurants:
                            if restaurant.getName() == Course[0][idx]:
                                Course[0][idx] = restaurant

                    case "카페":
                        for cafe in Cafes:
                            if cafe.getName() == Course[0][idx]:
                                Course[0][idx] = cafe

                    case "놀거리":
                        for doing_fun in Doing_funs:
                            if doing_fun.getName() == Course[0][idx]:
                                Course[0][idx] = doing_fun

                line.append(Course[0][idx].getLocation())
            rep = str(n)+"번째 추천 경로"
            routes[rep] = line
            n+=1

        colors = ['red', 'blue', 'green', 'orange', 'purple']

        M = folium.Map(location=routes['1번째 추천 경로'][0], zoom_start=9)  #1번째 추천 경로의 첫 번째 위치를 기준으로 함.

        for (name, route), color in zip(routes.items(), colors):
            folium.PolyLine(locations=route, color = color).add_to(M)

        for i, (name, route) in enumerate(routes.items()):
            for j, coord in enumerate(route):
                rep = f"{name} - {j+1}번째 위치"
                match separted_date[j]:
                    case "식사":
                        rep += " (식당)"

                    case "카페":
                        rep += " (카페)"

                    case "놀거리":
                        rep += " (놀거리)"

                folium.Marker(location=coord, tooltip=rep).add_to(M)


        M.save("map.html")  # map API 사용하여 위치 정보들과 경로들 시각화함.
        webbrowser.open("map.html")

        req_detail = "0"
        
        while(req_detail != "종료"):
            req_detail = input("몇 번째 추천 경로의 위치 정보들을 더 자세히 보고 싶으신가요?(space bar가 포함되면 안 됩니다.) \n(2번째 추천 경로를 상세히 보고 싶다면) ex)2  \n-------------------  \n⋇종료를 원하시면, ex)종료 \n: ")
            match req_detail:
                case "1":
                    Details = Full_course_candidates[int(req_detail)-1][0]

                case "2":
                    Details = Full_course_candidates[int(req_detail)-1][0]

                case "3":
                    Details = Full_course_candidates[int(req_detail)-1][0]

                case "4":
                    Details = Full_course_candidates[int(req_detail)-1][0]

                case "5":
                    Details = Full_course_candidates[int(req_detail)-1][0]

                case "종료":
                    break

                case _:
                    req_detail = input("잘못 입력하셨습니다. 다시 입력해주세요! \n몇 번째 추천 경로의 위치 정보들을 더 자세히 보고 싶으신가요?(⋇ space bar가 포함되면 안 됩니다.) \n(2번째 추천 경로를 상세히 보고 싶다면) ex)2  \n-------------------  \n⋇종료를 원하시면, ex)종료 \n: ")
                    continue
            
            print("-----------------------------------------------------")
            print(req_detail+"번째 추천 경로에 대한 추가 정보들입니다!")
            print("-----------------------------------------------------\n")

            idx = 1
            for place in Details:
                rep = "     "
                rep += str(idx)+"번째 위치 정보 : "+"이름 : {:}, 분류 : {:}, 평점 : {:}, 운영시간 : {:}, ".format(place.getName(), place.getType(), place.getRating(), place.operating_tiem)
                match place.getType():
                    case "식당":
                        rep += "음식 종류 : {:}, 대표메뉴 : {:}, 평균가 : {:}".format(place.getTypeOfFood(), place.getRepMenu(), place.getAvgPrice())

                    case "카페":
                        rep += "커피 말고 마실 만한 메뉴가 있는 지 : {:}, 대표메뉴 : {:}".format(place.getNoCoffeePossible(), place.getRepMenu())
                        if not coffee and place.getNoCoffeePossible() == "O":
                            rep = rep[:14]+"(Recommend!)"+rep[14:]

                    case "놀거리":
                        rep += "공연 여부 : {:}, 공연 정보 : {:}".format(place.getPerformanceAvailable(), place.getEntertainmentsInfo())
                

                print(rep)
                print()

                idx += 1

            print("-"*100)
            print()
            
        command = input("다시 시작하기를 원하신다면 '시작'을, 종료를 원하신다면 '종료'를 입력해주세요! \n ex)종료")

                


                

                         데이트 코스  예상 도보 시간(h)(3km/h기준)
1번째 추천 :    식당4 -> 카페6 -> 놀이공원4              4.832286
2번째 추천 :    식당2 -> 카페9 -> 놀이공원8             10.315266
3번째 추천 :    식당7 -> 카페7 -> 놀이공원8             20.269768
4번째 추천 :   식당10 -> 카페4 -> 놀이공원6             12.049917
5번째 추천 :   식당4 -> 카페10 -> 놀이공원9              7.509030
-----------------------------------------------------
1번째 추천 경로에 대한 추가 정보들입니다!
-----------------------------------------------------

     1번째 위치 정보 : 이름 : 식당4, 분류 : 식당, 평점 : 3.5, 운영시간 : 10:00~20:03, 음식 종류 : 일식, 대표메뉴 : 초밥, 평균가 : 10000

     2번째 위치 정보(Recommend!) : 이름 : 카페6, 분류 : 카페, 평점 : 4.5, 운영시간 : 10:00~20:05, 커피 말고 마실 만한 메뉴가 있는 지 : O, 대표메뉴 : 녹차빙수

     3번째 위치 정보 : 이름 : 놀이공원4, 분류 : 놀거리, 평점 : 3, 운영시간 : 10:00~20:03, 공연 여부 : O, 공연 정보 : url

----------------------------------------------------------------------------------------------------

-----------------------------------------------------
2번째 추천 경로에 대한 추가 정보들입니다!
---------------------------------------------------

## (8) 결론
* 비록 가상의 위치 정보일지라도, 이것이 현실의 데이터로 대체되었을 때, 서울이 익숙하지 않은 사람에게 도움이 될 것이라는 생각을 한다.
* 최근접 이웃 알고리즘은 본전공의 '알고리즘과 수학'이라는 과목에서 salesman problem을 접하며 공부하게 되었고, 비록 시간 복잡도가 증가하더라도, <br>완전 연결 그래프인 상황이라서 다익스트라 알고리즘보다 최근접 이웃 알고리즘이 더 적합할 것 같다는 생각에 사용하게 되었고, 중복 제거 등의 기능도 더 쉽게 구현 가능해졌다.
* folium, geodesic의 활용법은 chatGpt에게 예제코드와 설명을 요구하고, 인터넷을 뒤져가며 코드를 짜기 위해 공부했다.
* 만든 프로그램을 웹에 올리는 것이 목표였는데, 동적인 기능을 하게 하는 JavaScript 없이는 불가능하다고 생각해서, 웹에 올리는 것은 JavaScript를 공부한 뒤에 시도해보겠다.